# d=5

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import torch

from qecdec import RotatedSurfaceCode_Memory
from learned_decoders import *
from train_utils import *
from datetime import datetime

# Set experiment parameters.
d = 5  # code distance
rounds = 5  # number of rounds of stabilizer measurements
p = 0.01  # physical error rate

# Set training parameters.
num_epochs = 20
batch_size = 256
model_kwargs = dict(num_iters=5)
optimizer_kwargs = dict(lr=0.002)
loss_fn_kwargs = dict(beta=1.0, skip_iters=0)
lr_scheduler_kwargs=dict(factor=0.2, patience=3, threshold=1e-3, threshold_mode="abs")
early_stopper_kwargs = dict(patience=5, min_delta=1e-3)

In [2]:
expmt = RotatedSurfaceCode_Memory(
    d=d,
    rounds=rounds,
    basis='Z',
    data_qubit_error_rate=p,
    meas_error_rate=p,
)
print("Number of error mechanisms:", expmt.num_error_mechanisms)
print("Number of detectors:", expmt.num_detectors)
print("Number of observables:", expmt.num_observables)

Number of error mechanisms: 186
Number of detectors: 72
Number of observables: 1


In [3]:
train_dataset, val_dataset = build_datasets(
    expmt,
    train_shots=10_000,
    val_shots=1_000,
    seed=42,
    train_all_wt1_errors=True,
    train_all_wt2_errors=True,
    remove_trivial_syndromes=True,
)

model = LearnedDMemBP(expmt.chkmat, expmt.prior, **model_kwargs)
loss_fn = DecodingLoss(expmt.chkmat, expmt.obsmat, **loss_fn_kwargs)
metric = DecodingMetric(expmt.chkmat, expmt.obsmat)
optimizer = torch.optim.Adam(model.parameters(), **optimizer_kwargs)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, **lr_scheduler_kwargs)
early_stopper=EarlyStopper(**early_stopper_kwargs)

Sampling shots from the noisy circuit...
Added 8165 samples to the training dataset.
Added 815 samples to the validation dataset.
Generating all weight-1 errors...
Added 186 samples to the training dataset.
Generating all weight-2 errors...
Added 17205 samples to the training dataset.
Size of train_dataset: 25556
Size of val_dataset: 815


In [4]:
train_decoder(
    model,
    train_dataset,
    val_dataset,
    loss_fn,
    metric,
    optimizer,
    num_epochs=num_epochs,
    batch_size=batch_size,
    device="cpu",
    lr_scheduler=lr_scheduler,
    early_stopper=early_stopper,
)

Using cpu device


Epoch 1/20: 100%|██████████| 100/100 [00:16<00:00,  6.04it/s, avg_loss=0.767078, grad_norm=0.303137]


Epoch 1 Summary:
  Avg Train Loss: 0.767078
  Avg Val Loss: 0.832159
  wrong_syndrome_rate: 0.050307
  wrong_observable_rate: 0.007362
  failure_rate: 0.050307
  Learning Rate: 0.002000



Epoch 2/20: 100%|██████████| 100/100 [00:16<00:00,  6.11it/s, avg_loss=0.704019, grad_norm=0.272567]


Epoch 2 Summary:
  Avg Train Loss: 0.704019
  Avg Val Loss: 0.784984
  wrong_syndrome_rate: 0.050307
  wrong_observable_rate: 0.011043
  failure_rate: 0.050307
  Learning Rate: 0.002000



...

Epoch 10 Summary:
  Avg Train Loss: 0.635644
  Avg Val Loss: 0.750919
  wrong_syndrome_rate: 0.035583
  wrong_observable_rate: 0.012270
  failure_rate: 0.036810
  Learning Rate: 0.000400



Epoch 11/20: 100%|██████████| 100/100 [00:16<00:00,  6.04it/s, avg_loss=0.633803, grad_norm=0.062244]


Epoch 11 Summary:
  Avg Train Loss: 0.633803
  Avg Val Loss: 0.750666
  wrong_syndrome_rate: 0.034356
  wrong_observable_rate: 0.012270
  failure_rate: 0.035583
  Learning Rate: 0.000400



Epoch 12/20: 100%|██████████| 100/100 [00:16<00:00,  5.91it/s, avg_loss=0.633886, grad_norm=0.158321]


Epoch 12 Summary:
  Avg Train Loss: 0.633886
  Avg Val Loss: 0.750557
  wrong_syndrome_rate: 0.034356
  wrong_observable_rate: 0.012270
  failure_rate: 0.035583
  Learning Rate: 0.000400

Early stopping triggered


In [ ]:
gamma = torch.stack([x.detach() for x in model.gamma]).cpu().numpy().astype(np.float64)
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
np.save(f"learned_params/dmembp_d{d}_{timestamp}_gamma.npy", gamma)

# d=7

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import torch

from qecdec import RotatedSurfaceCode_Memory
from learned_decoders import *
from train_utils import *
from datetime import datetime

# Set experiment parameters.
d = 7  # code distance
rounds = 7  # number of rounds of stabilizer measurements
p = 0.01  # physical error rate

# Set training parameters.
num_epochs = 20
batch_size = 256
model_kwargs = dict(num_iters=5)
optimizer_kwargs = dict(lr=0.002)
loss_fn_kwargs = dict(beta=1.0, skip_iters=0)
lr_scheduler_kwargs=dict(factor=0.2, patience=3, threshold=1e-3, threshold_mode="abs")
early_stopper_kwargs = dict(patience=5, min_delta=1e-3)

In [2]:
expmt = RotatedSurfaceCode_Memory(
    d=d,
    rounds=rounds,
    basis='Z',
    data_qubit_error_rate=p,
    meas_error_rate=p,
)
print("Number of error mechanisms:", expmt.num_error_mechanisms)
print("Number of detectors:", expmt.num_detectors)
print("Number of observables:", expmt.num_observables)

Number of error mechanisms: 512
Number of detectors: 192
Number of observables: 1


In [3]:
train_dataset, val_dataset = build_datasets_version_2(
    expmt,
    train_shots=100_000,
    val_shots=10_000,
    seed=42,
)

model = LearnedDMemBP(expmt.chkmat, expmt.prior, **model_kwargs)
loss_fn = DecodingLoss(expmt.chkmat, expmt.obsmat, **loss_fn_kwargs)
metric = DecodingMetric(expmt.chkmat, expmt.obsmat)
optimizer = torch.optim.Adam(model.parameters(), **optimizer_kwargs)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, **lr_scheduler_kwargs)
early_stopper=EarlyStopper(**early_stopper_kwargs)

Sampling shots from the noisy circuit...
Added 13427 samples to the training dataset.
Added 1338 samples to the validation dataset.
Generating all weight-1 errors...
Added 512 samples to the training dataset.
Generating all weight-2 errors...
Added 2282 samples to the training dataset.
Size of train_dataset: 16221
Size of val_dataset: 1338


In [4]:
train_decoder(
    model,
    train_dataset,
    val_dataset,
    loss_fn,
    metric,
    optimizer,
    num_epochs=num_epochs,
    batch_size=batch_size,
    device="cpu",
    lr_scheduler=lr_scheduler,
    early_stopper=early_stopper,
)

Using cpu device


Epoch 1/20: 100%|██████████| 64/64 [00:31<00:00,  2.00it/s, avg_loss=8.084339, grad_norm=1.444346]


Epoch 1 Summary:
  Avg Train Loss: 8.084339
  Avg Val Loss: 7.735803
  wrong_syndrome_rate: 0.828849
  wrong_observable_rate: 0.110613
  failure_rate: 0.829596
  Learning Rate: 0.002000



Epoch 2/20: 100%|██████████| 64/64 [00:30<00:00,  2.09it/s, avg_loss=6.969124, grad_norm=1.287716]


Epoch 2 Summary:
  Avg Train Loss: 6.969124
  Avg Val Loss: 6.936110
  wrong_syndrome_rate: 0.724963
  wrong_observable_rate: 0.106876
  failure_rate: 0.725710
  Learning Rate: 0.002000



...

Epoch 18 Summary:
  Avg Train Loss: 4.896978
  Avg Val Loss: 5.331110
  wrong_syndrome_rate: 0.323617
  wrong_observable_rate: 0.076233
  failure_rate: 0.323617
  Learning Rate: 0.002000



Epoch 19/20: 100%|██████████| 64/64 [00:30<00:00,  2.09it/s, avg_loss=4.902651, grad_norm=0.689222]


Epoch 19 Summary:
  Avg Train Loss: 4.902651
  Avg Val Loss: 5.330956
  wrong_syndrome_rate: 0.325112
  wrong_observable_rate: 0.078475
  failure_rate: 0.325112
  Learning Rate: 0.002000



Epoch 20/20: 100%|██████████| 64/64 [00:30<00:00,  2.11it/s, avg_loss=4.900708, grad_norm=0.742198]


Epoch 20 Summary:
  Avg Train Loss: 4.900708
  Avg Val Loss: 5.325924
  wrong_syndrome_rate: 0.325112
  wrong_observable_rate: 0.079223
  failure_rate: 0.325112
  Learning Rate: 0.002000



In [5]:
gamma = torch.stack([x.detach() for x in model.gamma]).cpu().numpy().astype(np.float64)
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
np.save(f"learned_params/dmembp_d{d}_{timestamp}_gamma.npy", gamma)